#PYTHON we scrapping project

#scrapping web data using Python


extracting data from valorant(game) forum website vlr.gg

1.   scrapping the thread data from vlr.gg(website)
2.   tools

    *   python
    *   requests: to download the webpage
    *   BeautifulSoup: to parse the HTML 
    *   Pandas: to handle the data and convert it into different(desired) format








#Here are the steps to follow
 * we are gonna scrape the url https://vlr.gg/threads
 * we are gonna extract the different thread title,author,author url,post comment,post url and post text
 *then we will save all the data in csv 

#Use the request library to download webpages

#importing all the Libraries

importing all the libraries


In [86]:
#import all the libraries
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re 

#Creating a thread Extract function

in thread extract function 
  * we have defined an empty dictionary with the name of all the attributes we want to extract
  * using requests downloading the webpage
  *then using beautifulsoup to parse the html
  * using beautifulsoup we have defined , the title tag,date,comment,and base url
  * using a loop to save all the data in dictionary dict2

In [115]:
# extracting title ,date,comments ,post url from the thread page of Vlr.gg 

def thread_extract(topic_url):
    dict2={'title':[],'date_posted':[],'comments':[],'post_url':[],'author':[],'author_url':[],'post_text':[]}
    #initiating the response checking for status code
    response=requests.get(topic_url)
    #if status code not equals to 200 return
    if response.status_code !=200:
        return
    #parsing html using beautiful soup
    doc=BeautifulSoup(response.text,'html.parser')
    #title anchor tag 
    title_a_tag=doc.find_all('a',{'class':'thread-item-header-title'})
    #parsing date posted
    date_posted=doc.find_all('span',{'class':"date-full hide"})
    #parsing the total comments
    comment=doc.find_all('span',{'class':'post-count'})
    #extracting the post_url
    post_url=doc.find_all('a',{'class':"thread-item-header-title"})
    base_url='https://vlr.gg'
    i=0;
    while i<len(title_a_tag):
        dict2['title'].append(title_a_tag[i].text.strip())

        dict2['comments'].append(re.findall('\d+',comment[i].text)) #using regular expression to extract the number from comments
        
        dict2['date_posted'].append(date_posted[i].text.strip())

        dict2['post_url'].append(base_url+post_url[i]['href'])
        i+=1
    return dict2



    

#Creating a helper function to extract autor name ,author url and post detail

using the different post url contained in dictionary we will download the webpages of different url and parse them using beautifulSoup
then we will save the data in dictionary using while loops 

**note the code is not well optimised**

In [ ]:
#creating the helper function to extract the data 
def get_post_details(response):
    #defining the variables
    i=j=k=0
    #initialising the Beautiful soup for all 
    post_doc=BeautifulSoup(response.text,'html.parser')
    author=post_doc('a',{'class':'post-header-author'})
    base_url='https://vlr.gg'
    div_post_body=post_doc.find_all('div',{'class':"post-body"})
    author_name=author[0].text.strip()
    author_url=base_url+author[0]['href']
    post_text=[]
    while i < len(div_post_body):
        while j < len(div_post_body[i]('p')):
            post_text.append(div_post_body[i]('p')[j].text.strip())
            j+=1
        while k < len(div_post_body[i]('a')):
            post_text.append(div_post_body[i]('a')[k]['href'])
            k+=1
        i+=1
    return [author_name,author_url,post_text]

#using helper function in our main Post_extract function

using a loop saving the data in dictionary

In [145]:
def post_extract(dict2):
    #    dict2={'title':[],'date_posted':[],'comments':[],'post_url':[],'author':[],'author_url':[],'post_text':[]}
    #getting the webpage
    x=0
    while x < len(dict2['post_url']):
    # downloading all the page url in loop
        response=requests.get(dict2['post_url'][x])
    #parsing using BeautifulSoup
        dict2["author"].append(get_post_details(response)[0])
        dict2["author_url"].append(get_post_details(response)[1])
        dict2["post_text"].append(get_post_details(response)[2])
        x+=1
    # returning th dictionary containing author,author_url,and post_text
    return dict2


#Saving to CSV

In [158]:
#saving it as csv
def to_csv(dict2):
    count=0
    df=pd.DataFrame(dict2)
    df.to_csv('page{}.csv'.format(count),index=None)

#using the functions

In [153]:
topic_url=input('place the link: ')
a=thread_extract(topic_url)
data=post_extract(a)
to_csv(data)


place the link: https://vlr.gg/threads
